<a href="https://colab.research.google.com/github/prairie-dogY/test/blob/main/%E8%89%AF%E5%93%81%E3%80%81%E7%95%B0%E5%B8%B8%E5%93%81%E3%81%AE%E6%A4%9C%E7%9F%A5_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

無料版　GPU　だと　CUDA　Errorが多発しますのでご注意ください

1 step モデルを作る
- 精度向上の改善方法を考える
  - 画像が足りない
  - 画像がわかりにくい
  - ハイパーパラメータ設定が適切でない
  - モデルが単調
- 精度向上を実装する
  - 画像のかさまし
  - 学習済みモデルの使用
  - テンサーボードで可視化

In [ ]:
!pip install -q pytorch-lightning
!pip install -q torchmetrics

     |████████████████████████████████| 925 kB 5.7 MB/s 
     |████████████████████████████████| 282 kB 46.0 MB/s 
     |████████████████████████████████| 829 kB 35.6 MB/s 
     |████████████████████████████████| 125 kB 43.9 MB/s 
     |████████████████████████████████| 636 kB 41.1 MB/s 
     |████████████████████████████████| 1.3 MB 34.6 MB/s 
     |████████████████████████████████| 160 kB 42.5 MB/s 
     |████████████████████████████████| 294 kB 46.1 MB/s 


In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torchvision import transforms, datasets
import torchmetrics
from torchmetrics.functional import accuracy

## ベースモデルの作成

In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder('/content/drive/MyDrive/kikagaku/data/images_screw_data/train', transform)

# クラスを確認
dataset.class_to_idx

{'screw0_OK': 0, 'screw1_NG': 1}

In [ ]:
pl.seed_everything(0)
n_train = int(len(dataset) * 0.7)
n_val = len(dataset) - n_train
train, val = torch.utils.data.random_split(dataset, [n_train, n_val])
batch_size = 32
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=40)

class Net(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(56 * 56 * 3, 64)
        self.fc2 = nn.Linear(64, 2)
        self.bn = nn.BatchNorm2d(3)

    def forward(self, x):
        h = F.relu(self.bn(self.conv(x)))
        h = F.max_pool2d(h, 2, 2)
        h = F.relu(self.conv2(h))
        h = F.max_pool2d(h, 2, 2)
        h = h.view(h.size(0), -1)
        h = F.relu(self.fc1(h))
        h = self.fc2(h)
        return h

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        self.log('train_acc', accuracy(y.softmax(dim=-1), t), on_step=True, on_epoch=True)
        return loss

    # 検証データに対する処理
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer

net = Net()
trainer = pl.Trainer( max_epochs=10)
trainer.fit(net, train_loader, val_loader)

Global seed set to 0


In [ ]:
trainer.callback_metrics

NameError: ignored

##その他の考慮事項
Recall の算出  
推論時間の考慮

In [ ]:
net.eval()

Net(
  (conv): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=9408, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=2, bias=True)
  (bn): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
val_recall = torch.utils.data.DataLoader(val,40)
val_x,val_t = next(iter(val_recall ))

In [ ]:
y= net(val_x)
y = y.to('cpu').detach().numpy().copy()
val_t= val_t.to('cpu').detach().numpy().copy()

In [ ]:
from sklearn.metrics import recall_score

In [ ]:
y_pred = y.argmax(axis=1)
# scikit-learn で計算する場合
recall = recall_score(val_t, y_pred,average=None)
print(recall)
#{'screw0_OK': 0, 'screw1_NG': 1}

[0.6  0.55]


In [ ]:
%%time
y= net(val_x)
#0.134秒,人間に比べ遥かに推論が早い

CPU times: user 125 ms, sys: 3.97 ms, total: 129 ms
Wall time: 129 ms


## Efficient Net
300 epoch 回せる方は回してみてください

In [ ]:
!pip install efficientnet_pytorch
import os
import sys
sys.path.append("../input/efficientnetpytorch") #for efficient model

#for efficient model transfer learning
from efficientnet_pytorch import EfficientNet

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=1e4e0acd4e6dd4888f316660c551fda0002398d8597f6e65c0455baeb16bd88f
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
EfficientNet.from_pretrained('efficientnet-b0')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        40, 10, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        10, 40, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        40, 24, kernel_siz

In [ ]:
class Net(pl.LightningModule):

    def __init__(self):
        super().__init__()

        self.feature = EfficientNet.from_pretrained('efficientnet-b0')#b0~b6
        self.fc = nn.Linear(1000, 2)


    def forward(self, x):
        h = self.feature(x)
        h = self.fc(h)
        return h


    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        print(loss)
        self.log('train_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
        return loss


    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
        return loss


    # def test_step(self, batch, batch_idx):
    #     x, t = batch
    #     y = self(x)
    #     loss = F.cross_entropy(y, t)
    #     self.log('test_loss', loss, on_step=False, on_epoch=True)
    #     self.log('test_acc', accuracy(y.softmax(dim=-1), t), on_step=False, on_epoch=True)
    #     return loss


    def configure_optimizers(self):
        optimizer = torch.optim.RMSprop(self.parameters(), lr=0.00032083565570584314)#ベイズで出した値, Azure NC12Prom　GPU使用
        return optimizer

In [ ]:

net = Net()
trainer = pl.Trainer(max_epochs=300)
trainer.fit(net, train_loader, val_loader)


#300 epochs →Acc 95%

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Loaded pretrained weights for efficientnet-b3


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1304: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer"

  | Name    | Type         | Params
-----------------------------------------
0 | feature | EfficientNet | 12.2 M
1 | fc      | Linear       | 2.0 K 
-----------------------------------------
12.2 M    Trainable params
0         Non-trainable params
12.2 M    Total params
48.941    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning: The number of training samples (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


...

Validating: 0it [00:00, ?it/s]

tensor(0.0435, grad_fn=<NllLossBackward>)
tensor(0.0090, grad_fn=<NllLossBackward>)
tensor(0.0035, grad_fn=<NllLossBackward>)
tensor(0.0048, grad_fn=<NllLossBackward>)


Validating: 0it [00:00, ?it/s]

tensor(0.0049, grad_fn=<NllLossBackward>)
tensor(0.0062, grad_fn=<NllLossBackward>)
tensor(0.0027, grad_fn=<NllLossBackward>)
tensor(0.0010, grad_fn=<NllLossBackward>)


Validating: 0it [00:00, ?it/s]

In [ ]:
trainer.callback_metrics

{'train_acc': tensor(1.),
 'train_loss': tensor(0.0037),
 'val_acc': tensor(0.5333),
 'val_loss': tensor(1.7081)}

## 【発展】ベイズ最適化
メモリ乗り切らなかったので別で実装してます

In [ ]:
!pip install optuna

In [ ]:
import optuna


In [ ]:
class Net(pl.LightningModule):

    def __init__(self, lr=0.01,batch_size=14, optimizer = 'SDG'): # defaultを0．01に設定
        super().__init__()

        self.lr = lr# lrを引数に設定
        self.optimizer = optimizer#optimizerを引数に設定

        self.conv = EfficientNet.from_pretrained('efficientnet-b3')
        self.fc1 = nn.Linear(1000, 2)

        # 学習済みのパラメータを固定しない
        # for param in self.conv.parameters():
        #     param.requires_grad = False

        # train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
        # val_loader = torch.utils.data.DataLoader(val, batch_size)
        # test_loader = torch.utils.data.DataLoader(test , batch_size)

    def forward(self, x):
        x = self.conv(x)
        x = self.fc1(x)
        return x

    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', accuracy(y, t), on_step=True, on_epoch=True, prog_bar=True)
        return loss

    # 検証データに対する処理
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        loss = F.cross_entropy(y, t)
        self.log('val_loss', loss, on_step=False, on_epoch=True)
        self.log('val_acc', accuracy(y, t), on_step=False, on_epoch=True)
        return loss


    # # テストデータに対する処理
    # def test_step(self, batch, batch_idx):
    #     x, t = batch
    #     y = self(x)
    #     loss = F.cross_entropy(y, t)
    #     self.log('test_loss', loss, on_step=False, on_epoch=True)
    #     self.log('test_acc', accuracy(y, t), on_step=False, on_epoch=True)
    #     return loss

    def configure_optimizers(self):
        if self.optimizer == 'SDG':
          optimizer = torch.optim.SGD(self.parameters(), lr=self.lr)
          return optimizer
        elif self.optimizer == 'RMSprop':
          optimizer = torch.optim.RMSprop(self.parameters(), lr=self.lr)
          return optimizer
        else:
          optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
          return optimizer

In [ ]:
def objective(trial):

  lr = trial.suggest_loguniform('lr',1e-5, 1e-2)#0.00001,0.01
  batch_size = 32

  optimizer = trial.suggest_categorical('optimizer', ['SDG', 'RMSprop', 'Adam'])

  pl.seed_everything(0)
  net= Net(lr)

  trainer = pl.Trainer(max_epochs=50, deterministic=True)
  trainer.fit(net, train_loader, val_loader)
  result = trainer.callback_metrics['val_acc']

  return result

In [ ]:
# study オブジェクトを作成し、最適化を実行
#Colabだと時間がかかるor乗り切らないので仮想マシンやGPU環境での実行をおすすめします。
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-10-04 00:40:18,191] A new study created in memory with name: no-name-8f4bbd7f-5591-4280-aaeb-d7ff4fbea297
Global seed set to 0
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1304: UserWarning:

GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.


  | Name | Type         | Params
--------------------------------------
0 | conv | EfficientNet | 12.2 M
1 | fc1  | Linear       | 2.0 K 
--------------------------------------
12.2 M    Trainable params
0         Non-trainable params
12.2 M    Total params
48.941    Total estimated model params size (MB)


Loaded pretrained weights for efficientnet-b3


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning:

The number of training samples (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



Training: -1it [00:00, ?it/s]

...

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:327: UserWarning:

The number of training samples (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.



Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
# print(study.best_params)
# print(study.best_value)
# print(study.best_trial)

## Keras

In [ ]:
# from tensorflow.keras.callbacks import Callback
# def objective(trial):

#     K.clear_session()
#     tf.random.set_seed(0)

#     base_model = EfficientNetB0(include_top=False, weights='imagenet')

#     base_model.trainable = False

#     # モデルの構築
#     x = base_model.output
#     x = layers.GlobalAveragePooling2D()(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dense(640, activation='relu')(x)
#     x = layers.Dense(2, activation='softmax')(x)

#     model = models.Model(base_model.inputs, x)

#     # ハイパーパラメータ（最適化手法と学習率を調べる）
#     optimizer = trial.suggest_categorical('optimizer', ['sgd', 'rmsprop', 'adam'])
#     learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e0)

#     if optimizer == 'sgd':
#         model.compile(tf.keras.optimizers.SGD(learning_rate), 'sparse_categorical_crossentropy', metrics=['accuracy'])
#     elif optimizer == 'rmsprop':
#         model.compile(tf.keras.optimizers.RMSprop(learning_rate), 'sparse_categorical_crossentropy', metrics=['accuracy'])
#     elif optimizer == 'adam':
#         model.compile(tf.keras.optimizers.Adam(learning_rate), 'sparse_categorical_crossentropy', metrics=['accuracy'])

#     # 学習
#     hist = model.fit(
#         train_generator,
#         steps_per_epoch=14,
#         epochs=50,
#         verbose=2,
#         validation_data=val_generator,
#         validation_steps=6
#     )

#     return 1.0 - np.max(hist.history['val_accuracy'])


In [ ]:
# study = optuna.create_study()
# study.optimize(objective, n_trials=50)

In [ ]:
# print(study.best_params)
# print(study.best_value)
# print(study.best_trial)

In [ ]:
# K.clear_session()
# tf.random.set_seed(0)

# base_model = EfficientNetB0(include_top=False, weights='imagenet')

# base_model.trainable = False

# # モデルの構築
# x = base_model.output
# x = layers.GlobalAveragePooling2D()(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dense(640, activation='relu')(x)
# x = layers.Dense(2, activation='softmax')(x)

# model = models.Model(base_model.inputs, x)

# # モデルのコンパイル
# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00032083565570584314),
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy'])

# # 学習
# hist = model.fit(
#     train_generator,
#     steps_per_epoch=14,
#     epochs=300,
#     verbose=2,
#     validation_data=val_generator,
#     validation_steps=6
# )

 https://www.evernote.com/l/Asx1doIuioZCLro6Rybg5pXUB1bqbk3I3MUB/image.png

## データの拡張

https://qiita.com/shoheihoh/items/137141864394c3e4866b  
https://qiita.com/enbi/items/f84d253b79184c903c27
- epoch 頃にランダムに拡張される。同じ写真だが角度等が変わるので